# Battleships

## Planning

- standard version & extended version (if we have time)
    - standard
        - normal rules
        - 2 human players
        - basic display (jupyter output)
    - extended
        - graphics display (turtle?)
        - hard mode ("Salvo")
        - computer player
            - random guess
            - if last guess == hit, random guess within closer range...
        - different size board
        - different size ships(?)


Standard:

- board
    - 10x10
    - 4 boards (player_1_sea, player_1_radar, player_2_sea, player_2_radar)

- ships
    - number/size has to make sense for the board dimensions
    - standard:
        - Carrier (5)
        - Battelship (4)
        - Cruiser (3)
        - Submarine (3)
        - Destroyer (2)

- turn
    - guess one coordinate on board
        - outputs result (miss / hit + ship if hit)
            - sunk if ship gone
        - displays new board (updated pegs)

- display
    - can only see your board

Functions:

- display_board
    - input lists
    - output (print) sea and radar

- player_names
    - input string

- place_ships
    - choose ship
    - choose starting coordinate
    - choose direction (udlr)
    - calls check_space_free (iterates for all coords in selection)
    - updates ship variables

- update board

- check_space_free

- choose_first_player
    - random (0, 1)

- fire_shot
    - choose location to fire at
    - update board/list

- check_hit
    - returns result of shot
    - update ship count

- check_ship_status
    - checks if 

- save_shot
    - remove available firing locations
    - decide whether can play in same square again

- ready_for_turn
    - when turn ended, hide board
    - ask if ready
    - when ready, show other player board

- winning_condition
    - ship count == 0

- replay

In [25]:
print("\U0001F6E5")
print("\U000026F4")
print("\U0001F4A5")

🛥
⛴
💥


## Functions

In [180]:
# Library imports
import random
import IPython
from IPython.display import display, clear_output
import ipywidgets as widgets

In [71]:
def player_names():
    """Asks for the names of the players and returns both in a tuple."""
    player_1 = input("First name: ")
    player_2 = input("Second name: ")
    return player_1, player_2

def choose_first_player(player_1_name, player_2_name):
    """Randomly selects which player has the first turn. Returns one of the two player names that were 
    passed to the function."""
    if random.randint(0,1) == 0:
        return player_1_name
    else:
        return player_2_name

def display_boards(player_name, radar_lst, sea_lst):
    """Prints the radar_lst and sea_lst lists in the format of Battleships boards, with the letters list
    denoting the rows."""
        
    print(player_name, "\n")
    print("Radar zone\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(radar_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")
            
    print("\nSea zone\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(sea_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")

In [112]:
def update_board_placement(sea_lst, boat_coords, symbol):
    """Updates the board after the placement of a boat, taking a list of boat_coords as input."""
    for coord in boat_coords:
        sea_lst[coord] = symbol
        
def update_board_combat(radar_lst, sea_lst, position, flag):
    """Taking the position and flag results from a turn, this function updates the radar_lst and 
    sea_lst lists that make up the board."""
    if flag:
        radar_lst[position] = radar_hit
        sea_lst[position] = sea_hit
    else:
        radar_lst[position] = radar_miss
        sea_lst[position] = sea_miss

In [149]:
def enforce_player_coord(coord):
    """Ensures a player coordinate input is in the valid 'A1' - 'J10' format. Returns coord once in the correct format."""
    # Ensure second item in coord is a number
    try:
        int(coord[1])
    except (ValueError, IndexError):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()

    # Ensure coord is 2-3 characters long (and ends in '10') if 3 characters, starts with a letter in letters and
    # ends in a number 1-10
    while len(coord) > 3 or len(coord) < 2 or \
    (len(coord) == 3 and coord[1:3] != "10") or \
    coord[0] not in letters or int(coord[1]) not in range(1, 10):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()
     
    return coord.upper()

def coord_string_to_digit(coord):
    return letters.index(coord[0]) * 10 + int(coord[1:])

In [86]:
def place_boats(player_name, radar_lst, sea_lst, player_boat_coords):
    """Prompts user to place boats in descending order of length, by entering starting co-ordinate and direction.
    Calls check_space_free() to ensure boat placement is valid - if not, prompts user to place boat elsewhere.
    Saves placed boats in player_boats list."""
    
    for boat, data in boat_types.items():
        length, symbol = data
        coords = ""
        while not coords:            
            start_coord = coord_string_to_digit(enforce_player_coord(input(f"Enter start coordinate for {boat}: ").upper()))
            direction = ""
            while direction not in ["U","D","L","R"]:
                direction = input("Choose direction (U/D/L/R): ").upper()
            coords = check_space_free(sea_lst, start_coord, direction, length)
        player_boat_coords.append(coords)
        update_board_placement(sea_lst, coords, symbol)
        screen1.clear_output()
        with screen1:
            display(display_boards(player_name, radar_lst, sea_lst))
        screen1

In [ ]:
def check_space_free(sea_lst, start_coord, direction, length):
    """Checks if all required spaces for boat placement are free and in range.
    Returns coordinates if valid, otherwise returns False."""
    
    # use length of boat and direction to find all coords it will cover
    # if any of those coords are not sea empty, return False
        # also handle out of range issues
    # else return coords

    if sea_lst[start_coord] != sea_empty:
        return False
    
    boat_coords = [start_coord]
    row = start_coord//10
    if start_coord%10 == 0:
        row -= 1
                
    if direction in ["L", "R"]:
        d = 1
        if direction == "L":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * d)
            
            if (next_coord - 1) // 10 != row:
                return False
            elif sea_lst[next_coord] != sea_empty:
                print("Sea already occupied here")
                return False
            else:
                boat_coords.append(next_coord)
        return boat_coords
    
    elif direction in ["U", "D"]:
        d = 1
        if direction == "U":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * 10 * d)
            if next_coord not in range(1, 101):
                print("Ship outside board")
                return False
            elif sea_lst[next_coord] != sea_empty:
                print("Sea already occupied here")
                return False
            else:
                boat_coords.append(next_coord)
        return boat_coords
    
    else:
        return False

In [16]:
def fire_shot(player_guesses):
    """Calls enforce_player_coord() to prompt player to enter a guess and enforce validity.
    If coordinates have been guessed by the player already, asks for another guess.
    If the guess is valid, adds the guess to player_guesses and returns the 
    list index ('position') of the player guess."""
    
    guess = ""
    guess = enforce_player_coord(guess)
    
    # Checks if guess has already been guessed in a previous turn
    while guess in player_guesses:
        print("Coordinates already guessed!")
        guess = input("Enter coordinates (in the format 'A1'): ").upper() 
    
    player_guesses.append(guess) # adds guess to guess history
    position = letters.index(guess[0]) * 10 + int(guess[1:])
    return position

In [136]:
def check_hit(sea_lst, position):
    """Takes player guess position and returns True if that position in sea_lst was a boat."""
    if sea_lst[position] in sea_boats:
        return True
    else:
        return False

In [184]:
def check_ship_status(sea_lst, player_boat_coords, player_sunk_boats):
    """Checks which ships are destroyed and updates ship count."""
    for index, boat in enumerate(player_boat_coords):
        if all([sea_lst[boat[i]] == sea_hit for i in range(len(boat))]):
            player_sunk_boats.add(list(boat_types.keys())[index])
            
# Have running score on side?

In [116]:
def winning_condition():
    """Returns true if either player has 5 ships sunk."""
    if len(player_1_sunk_boats) == 5 or len(player_2_sunk_boats) == 5:
        return True
    else:
        return False

In [161]:
def new_turn(name):
    """When turn ended (i.e. function is called), hides the previous player's board and asks next player if they are ready.
    Once second player is ready, sets turn to their go and displays their board."""
    
    global turn
    screen1.clear_output()
    ready = input(f"{name}, enter Y to continue: ").upper()
    if ready == "Y":
        if name == player_1_name:
            with screen1:
                display(display_boards(name, radar_lst_p1, sea_lst_p1))
        else:
            with screen1:
                display(display_boards(name, radar_lst_p2, sea_lst_p2))
    else:
        pass
    turn = name

In [35]:
def final_result():
    """Displays both boards at the end of the game."""
    pass

In [59]:
replay_dropdown = widgets.Dropdown(
                    options=['Yes', 'No'],
                    value='Yes',
                    description='Replay? ',
                    disabled=False,
                )
def replay():
    """Asks players if they want to play again. If 'Yes', restarts game. If 'No', exits program."""
    if replay_dropdown.value == "Yes":
        return True
    else:
        return False
       
replay_dropdown

Dropdown(description='Replay? ', options=('Yes', 'No'), value='Yes')

## Game logic

Game Logic

- initiate empty board
- ask for player names
- decide who goes first

- player 1 places ships
    - display_board per ship placed
    - ready_for_turn
- player 2 places ships
    - display_board per ship placed
    - ready_for_turn

- while not winning_condition:
    - player 1 fire_shot
        - display board
        - (update board)
        - check_hit
        - check_ship_status
        - check winning_condition
        - display board
        - ready_for_turn
    - player 2 fire_shot (same logic as player 1)

- display result
    - display both boards (?)

- ask for replay

In [181]:
print("Welcome to Battleships!\n")

# Reset boards and define initial variables
letters = ["A","B","C","D","E","F","G","H","I","J"]

# Display containers
screen1 = widgets.Output()
screen2 = widgets.Output()
screen_container = widgets.HBox([screen1, screen2])
# screen1.clear_output()
display(screen1)

# Radar squares
radar_empty = "   "
radar_boats = [" D ", " C ", " S ", " B ", " A "]
radar_hit = " x "
radar_miss = " o "

# Sea squares
sea_empty = "~~~"
sea_boats = ["~D~", "~C~", "~S~", "~B~", "~A~"]
sea_hit = "~x~"
sea_miss = "~o~"

# Boat types
boat_types = {"Aircraft carrier (A)" : (5, "~A~"), 
              "Battleship (B)" : (4, "~B~"), 
              "Submarine (S)" : (3, "~S~"), 
              "Cruiser (C)" : (3, "~C~"), 
              "Destroyer (D)" : (2, "~D~")}

# List of player boats that have been sunk
player_1_sunk_boats = set()
player_2_sunk_boats = set()

# Lists of player boat locations
player_1_boat_coords = []
player_2_boat_coords = []

# To be included in game logic?
sea_lst_p1 = [sea_empty] * 101  
radar_lst_p1 = [radar_empty] * 101
sea_lst_p2 = [sea_empty] * 101  
radar_lst_p2 = [radar_empty] * 101

# Lists of past guesses
player_1_guesses = []
player_2_guesses = []


# Ask for player names
player_1_name, player_2_name = player_names()

# Determine which player starts
turn = choose_first_player(player_1_name, player_2_name)
print(f"\n{turn} starts!\n\n")

# Placing the boats
if turn == player_1_name:
    # Player 1 places ships
    with screen1:
        display_boards(player_1_name, radar_lst_p1, sea_lst_p1)
    place_boats(player_1_name, radar_lst_p1, sea_lst_p1, player_1_boat_coords)
    new_turn(player_2_name)
    # Player 2 places ships
    place_boats(player_2_name, radar_lst_p2, sea_lst_p2, player_2_boat_coords)
    new_turn(player_1_name)
else:
    # Player 2 places ships
    with screen1:
        display_boards(player_2_name, radar_lst_p2, sea_lst_p2)
    place_boats(player_2_name, radar_lst_p2, sea_lst_p2, player_2_boat_coords)
    new_turn(player_1_name)
    # Player 1 places ships
    place_boats(player_1_name, radar_lst_p1, sea_lst_p1, player_1_boat_coords)
    new_turn(player_2_name)

# Taking turns to fire
while not winning_condition():
    if turn == player_1_name:
    # Player 1 turn to fire
        # Get index position of player guess
        position = fire_shot(player_1_guesses)
        # Check if player guess was a hit
        flag = check_hit(sea_lst_p2, position)
        # Update Player 1 radar and Player 2 sea with result of shot
        update_board_combat(radar_lst_p1, sea_lst_p2, position, flag)
        # Check if any of Player 2's ships sunk, update sunk_ships list 
        check_ship_status(sea_lst_p2, player_2_boat_coords, player_2_sunk_boats)
        # Check if winning condition fulfilled
        if winning_condition():
            with screen1:
                display(final_result())
            break
        # Display updated Player 1 boards
        with screen1:
            display_boards(player_1_name, radar_lst_p1, sea_lst_p1)

        # Call new_turn() to initiate change of turn
        ready = ""
        while ready != "Y":
            ready = input(f" Ready to pass to {player_2_name}? ").upper()
        new_turn(player_2_name)

    else:
    # Player 2 turn to fire
        # Get index position of player guess
        position = fire_shot(player_2_guesses)
        # Check if player guess was a hit
        flag = check_hit(sea_lst_p1, position)
        # Update Player 2 radar and Player 1 sea with result of shot
        update_board_combat(radar_lst_p2, sea_lst_p1, position, flag)
        # Check if any of Player 1's ships sunk, update sunk_ships list 
        check_ship_status(sea_lst_p1, player_1_boat_coords, player_1_sunk_boats)
        # Check if winning condition fulfilled
        if winning_condition():
            with screen1:
                display(final_result())
            break
        # Display updated Player 2 boards
        with screen1:
            display_boards(player_2_name, radar_lst_p2, sea_lst_p2)

        # Call new_turn() to initiate change of turn
        ready = ""
        while ready != "Y":
            ready = input(f" Ready to pass to {player_1_name}? ").upper()
        new_turn(player_1_name)

Welcome to Battleships!



Output()

First name: K
Second name: J

J starts!


Enter start coordinate for Aircraft carrier (A): A1
Choose direction (U/D/L/R): r
Enter start coordinate for Battleship (B): B1
Choose direction (U/D/L/R): r
Enter start coordinate for Submarine (S): C1
Choose direction (U/D/L/R): r
Enter start coordinate for Cruiser (C): D1
Choose direction (U/D/L/R): r
Enter start coordinate for Destroyer (D): E1
Choose direction (U/D/L/R): r
K, enter Y to continue: Y
Enter start coordinate for Aircraft carrier (A): A10
Choose direction (U/D/L/R): l
Enter start coordinate for Battleship (B): b10
Choose direction (U/D/L/R): l
Enter start coordinate for Submarine (S): c10
Choose direction (U/D/L/R): l
Enter start coordinate for Cruiser (C): d10
Choose direction (U/D/L/R): l
Enter start coordinate for Destroyer (D): e10
Choose direction (U/D/L/R): l
J, enter Y to continue: Y
Try Error - Enter coordinates (in the format 'A1'): C8
 Ready to pass to K? Y
K, enter Y to continue: Y
Try Error - Enter coordinates (in t

In [183]:
# screen1
player_1_sunk_boats

['Submarine (S)',
 'Submarine (S)',
 'Submarine (S)',
 'Submarine (S)',
 'Submarine (S)']

In [185]:
# print error messages
# print hit/miss message
# manage(clear) text output

# make sure display in right place
# replay function